In [2]:
from utils import login, get_atm_strike, get_option_symbols, get_quotes
from config import Config
from logger_setup import app_logger
config = Config('creds/config.yaml', 'creds/tbs_rules.yaml')
api = login(config)


In [4]:

async def _get_atm_strike():
    try:
        atm_strike = await get_atm_strike(
            config.get_rule('tsymbol'), 
            lambda e, t: api.get_quotes(e, t)
        )
        if not atm_strike:
            app_logger.error("Failed to get ATM strike. Aborting simulation.")
            return None
        return atm_strike
    except Exception as e:
        app_logger.error(f"Error getting ATM strike: {e}")
        return None

async def _get_option_symbols(atm_strike):
    try:
        strikes = {
            'sce': (atm_strike + config.get_rule('sotm_points') + config.get_rule('bias_points'), 'CE'),
            'spe': (atm_strike - config.get_rule('sotm_points') + config.get_rule('bias_points'), 'PE'),
            # 'bce': (atm_strike + config.get_rule('botm_points') + config.get_rule('bias_points'), 'CE'),
            # 'bpe': (atm_strike - config.get_rule('botm_points') + config.get_rule('bias_points'), 'PE'),
        }
        option_symbols = await get_option_symbols(config.get_rule('tsymbol'), strikes)
        if not option_symbols:
            app_logger.error("Failed to get option symbols. Aborting simulation.")
            return None
        return option_symbols
    except Exception as e:
        app_logger.error(f"Error getting option symbols: {e}")
        return None

In [5]:
atm_strike = await _get_atm_strike()
option_symbols = await _get_option_symbols(atm_strike)

In [6]:
option_symbols

{'sce': {'Exchange': 'NFO',
  'Token': 48458,
  'LotSize': 15,
  'Symbol': 'BANKNIFTY',
  'TradingSymbol': 'BANKNIFTY11SEP24C51400',
  'Expiry': Timestamp('2024-09-11 00:00:00'),
  'Instrument': 'OPTIDX',
  'OptionType': 'CE',
  'StrikePrice': 51400.0},
 'spe': {'Exchange': 'NFO',
  'Token': 48459,
  'LotSize': 15,
  'Symbol': 'BANKNIFTY',
  'TradingSymbol': 'BANKNIFTY11SEP24P51400',
  'Expiry': Timestamp('2024-09-11 00:00:00'),
  'Instrument': 'OPTIDX',
  'OptionType': 'PE',
  'StrikePrice': 51400.0}}

In [7]:
from margin_calculator import MarginCalculator
margin_calculator = MarginCalculator(api, config.get_rule('account_id'))
adjusted_quantity = 50  # Specify the quantity you want to calculate the margin for
margin = await margin_calculator.calculate_margin(option_symbols, adjusted_quantity)
final_margin, final_trade_margin = margin
print(f'Margin: {final_margin}')
print(f'Trade Margin: {final_trade_margin}')

Margin: 311854.25
Trade Margin: 311854.25


In [14]:
print(margin)

None


In [3]:
from NorenRestApiPy.NorenApi import position

positionlist = []

position1 = position()
position1.prd = 'I'
position1.exch='NFO'
position1.instname = 'OPTIDX'
position1.symname = 'BANKNIFTY'
position1.exd= '11-SEP-2024'
position1.optt= 'CE'
position1.strprc= '51500.0'
position1.buyqty= '0'
position1.sellqty= '15'
position1.netqty= '-15'
positionlist.append(position1)

# position2 = position()
# position2.prd = 'I'
# position2.exch='NFO'
# position2.instname = 'OPTSTK'
# position2.symname = 'BANKNIFTY'
# position2.exd= '04-SEP-2024'
# position2.optt= 'PE'
# position2.strprc= '51200.0'
# position2.buyqty= '0'
# position2.sellqty= '15'
# position2.netqty= '-15'
# positionlist.append(position2)

# position3= position()
# position3.prd = 'I'
# position3.exch='NFO'
# position3.instname = 'OPTSTK'
# position3.symname = 'BANKNIFTY'
# position3.exd= '04-SEP-2024'
# position3.optt= 'CE'
# position3.strprc='50700.0'
# position3.buyqty= '15'
# position3.sellqty= '0'
# position3.netqty= '15'
# positionlist.append(position3)

# position4= position()
# position4.prd = 'I'
# position4.exch='NFO'
# position4.instname = 'OPTIDX'
# position4.symname = 'BANKNIFTY'
# position4.exd= '04-SEP-2024'
# position4.optt= 'PE'
# position4.strprc= '50700.0'
# position4.buyqty= '15'
# position4.sellqty= '0'
# position4.netqty= '115'
# positionlist.append(position4)

actid = 'xyz'
senddata = {}
senddata['actid'] =actid
senddata['pos'] = positionlist
#payload = 'jData=' + json.dumps(senddata,default=lambda o: o.encode())
#print(payload)
ret = api.span_calculator(actid,positionlist)
print(ret)

{'request_time': '09:34:47 06-09-2024', 'stat': 'Ok', 'span': '167556.60', 'expo': '30808.47', 'span_trade': '167556.60', 'expo_trade': '30808.47', 'pre_trade': '0.00', 'add': '0.00', 'add_trade': '0.00', 'ten': '0.00', 'ten_trade': '0.00', 'del': '0.00', 'del_trade': '0.00', 'spl': '0.00', 'spl_trade': '0.00'}


In [81]:
span = float(ret.get('span', 0))
expo = float(ret.get('expo', 0))
span_trade = float(ret.get('span_trade', 0))
expo_trade = float(ret.get('expo_trade', 0))

margin = span + expo
trade_margin = span_trade + expo_trade
final_margin = round(margin * 1.009, 2)
final_trade_margin = round(trade_margin * 1.009, 2)
print(f"Required margin: {final_trade_margin}")
print(f"Required trade margin: {final_trade_margin}")